# Imports

In [1]:
import numpy as np
import pandas as pd
import re


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import glob  # For loading multiple files

# import random
import os
import torch.optim as optim
from tqdm import tqdm
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from plotting import plot_stations_matplotlib

# from processAllData import getAllReadyForStationByLatAndLongAndK, getAllReadyForStationByLatAndLongAndKSplitTestAndTrain
# from LSTMArchitecture import GHIDataset, Main_LSTM
from constants import COLUMN_NAMES, USECOLS_NON_CLOUD, DTYPE_NON_CLOUD

%load_ext autoreload
%autoreload 2

# Functions

In [ ]:
def engineer_features(data):
    """
    Converts wind speed/direction into vector components,
    computes distance and alignment to station.
    """

    # Wind vector components
    data["wind_u"] = (
        data[COLUMN_NAMES["WIND_SPEED"]] * data[COLUMN_NAMES["WIND_DIRECTION_COS"]]
    )
    data["wind_v"] = (
        data[COLUMN_NAMES["WIND_SPEED"]] * data[COLUMN_NAMES["WIND_DIRECTION_SIN"]]
    )

    # Distance from sensor to station
    data["distance"] = np.sqrt(data["distanceX"] ** 2 + data["distanceY"] ** 2)

    # Alignment: how much wind points toward the station
    dot = data["wind_u"] * data["distanceX"] + data["wind_v"] * data["distanceY"]
    norm_wind = np.sqrt(data["wind_u"] ** 2 + data["wind_v"] ** 2)
    norm_station = np.sqrt(data["distanceX"] ** 2 + data["distanceY"] ** 2)
    data["alignment"] = dot / (
        norm_wind * norm_station + 1e-6
    )  # epsilon to avoid div by 0

    return data


def prepare_features(data):
    features = [COLUMN_NAMES["WIND_SPEED"], "wind_u", "wind_v", "distance", "alignment"]
    X = data[features]
    y = data[COLUMN_NAMES["GHI"]]
    return X, y


def train_distance_model(X_train, y_train):
    model = GradientBoostingRegressor(
        n_estimators=200, learning_rate=0.1, max_depth=4, random_state=42
    )
    model.fit(X_train, y_train)
    return model


def evaluate_distance_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"RMSE on test set: {rmse:.2f} W/m^2")

    # Feature importance
    importances = pd.Series(
        model.feature_importances_, index=X_test.columns
    ).sort_values(ascending=False)
    print("Feature importances:")
    print(importances)

In [ ]:
import glob  # For loading multiple files
import os
import re

import numpy as np
import pandas as pd
import torch

# from plotting import plot_stations_matplotlib
from constants import COLUMN_NAMES, DTYPE_NON_CLOUD, USECOLS_NON_CLOUD


def getStationTimePeriodFromYears(csv_file):
    match = re.search(r"(\d{4})-(\d{4})", csv_file)
    if match:
        start_year = int(match.group(1))
        end_year = int(match.group(2))
        # print(start_year, end_year)  # Output: 2005 2017
        return (start_year, end_year)


def denormalizeHourOrMonth(anglesin, anglecos, cycleLength):
    angle = np.arctan2(anglesin, anglecos)
    cycleBeforeModulo = (angle / (2 * np.pi)) * cycleLength + 1
    return cycleBeforeModulo % cycleLength


# Euclidean distance function
def euclidean_distance(lat1, lon1, lat2, lon2):
    distance_vector = ((lat2 - lat1), lon2 - lon1)
    return ((np.sqrt((lat2 - lat1) ** 2) + (lon2 - lon1) ** 2), distance_vector)


# Function to find the K nearest stations (not returning nearest station given)
def find_k_nearest_stations(df, station_name, k):
    if station_name not in df["station"].values:
        return f"Station {station_name} not found in the dataset."

    # Get the coordinates of the given station
    station_coords = df[df["station"] == station_name][
        ["Latitude", "Longitude"]
    ].values[0]
    lat1, lon1 = station_coords

    # Calculate the distance from the given station to all other stations
    df["distance"] = df.apply(
        lambda row: euclidean_distance(lat1, lon1, row["Latitude"], row["Longitude"]),
        axis=1,
    )

    # station_coords = df[df['station'] == station_name][['norm_lat', 'norm_long']].values[0]
    # lat1, lon1 = station_coords

    # # Calculate the distance from the given station to all other stations
    # df['distance'] = df.apply(lambda row: euclidean_distance(lat1, lon1, row['norm_lat'], row['norm_long']), axis=1)

    # Sort the DataFrame by distance and get the K nearest stations
    # nearest_stations = df[df['station'] != station_name].sort_values(by='distance').head(k)
    nearest_stations = (
        df[df["station"] != station_name]
        .sort_values(by="distance", key=lambda x: x.apply(lambda y: y[0]))
        .head(k)
    )

    return nearest_stations[
        ["station", "norm_lat", "norm_long", "distance", "StartTime", "EndTime"]
    ]


def latlon_to_xy(lat, lon, lat0):
    """Simple equirectangular projection."""
    R = 6371  # radius in km
    x = R * np.radians(lon) * np.cos(np.radians(lat0))
    y = R * np.radians(lat)
    return x, y


def compute_angle(vec1, vec2):
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    if norm1 == 0 or norm2 == 0:
        return 0  # Could return np.pi to maximize diversity if desired
    unit1 = vec1 / norm1
    unit2 = vec2 / norm2
    return np.arccos(np.clip(np.dot(unit1, unit2), -1.0, 1.0))


def makeDistanceTuple(distance, x, y):
    return (distance, (x, y))


def getEastWestNorthSouthOrder(result_df):
    ordered_rows = []

    # Make a copy so we can remove selected stations
    df_copy = result_df.copy()

    # 1. Most East
    east_idx = df_copy["x"].idxmax()
    ordered_rows.append(df_copy.loc[east_idx])
    df_copy = df_copy.drop(east_idx)

    # 2. Most West
    west_idx = df_copy["x"].idxmin()
    ordered_rows.append(df_copy.loc[west_idx])
    df_copy = df_copy.drop(west_idx)

    # 3. Most North
    north_idx = df_copy["y"].idxmax()
    ordered_rows.append(df_copy.loc[north_idx])
    df_copy = df_copy.drop(north_idx)

    # 4. Most South
    if not df_copy.empty:
        south_idx = df_copy["y"].idxmin()
        ordered_rows.append(df_copy.loc[south_idx])
        df_copy = df_copy.drop(south_idx)

    # Convert back to DataFrame
    return pd.DataFrame(ordered_rows).reset_index(drop=True)


def spatially_diverse_knn(df, station_name, k=3, candidate_pool=78):
    # spatially_diverse_knn_df
    #
    # This function selects the 'k' nearest stations to a given target station,
    # while ensuring that the selected neighbors are spatially diverse. Instead
    # of simply choosing the closest stations, it incorporates an angular diversity
    # criterion that selects stations spread out in different directions. This helps
    # avoid picking stations that are clustered in one region around the target.
    #
    # Parameters:
    #   - df: DataFrame containing station information (latitude, longitude, station_name)
    #   - center_latlon: Coordinates (latitude, longitude) of the target station
    #   - k: Number of neighbors to select (default is 3)
    #   - candidate_pool: Number of candidates to consider before selection (default is len(csv_files)=78)
    #
    # Returns:
    #   - A DataFrame with 'k' stations selected based on spatial diversity from the target
    #   - The coordinates of the target station
    # Exclude the center point if it exists in the set
    if station_name not in df["station"].values:
        return f"Station {station_name} not found in the dataset."

    center_latlon = df[df["station"] == station_name][["Latitude", "Longitude"]].values[
        0
    ]
    mask = ~(
        (df["Latitude"] == center_latlon[0]) & (df["Longitude"] == center_latlon[1])
    )
    df_filtered = df[mask].copy()
    # df_filtered['distance_vector'] = [(0.0, (0.0, 0.0))] * len(df_filtered)  # Initialize with default values

    # Convert to x/y coords
    lat0 = center_latlon[0]
    df_filtered["x"], df_filtered["y"] = latlon_to_xy(
        df_filtered["Latitude"], df_filtered["Longitude"], lat0
    )
    center_x, center_y = latlon_to_xy(center_latlon[0], center_latlon[1], lat0)
    center_xy = np.array([center_x, center_y])

    # Compute distances
    df_filtered["distance"] = np.linalg.norm(
        df_filtered[["x", "y"]].values - center_xy, axis=1
    )
    df_sorted = df_filtered.sort_values("distance").head(candidate_pool)
    df_sorted["distance"] = df_sorted.apply(
        lambda row: makeDistanceTuple(row["distance"], row["x"], row["y"]), axis=1
    )
    # print("After sorting by distance")
    # print(df_sorted.head())
    selected_rows = []
    selected_xy = []

    for idx, row in df_sorted.iterrows():
        cand_xy = np.array([row["distance"][1][0], row["distance"][1][1]])
        if len(selected_xy) == 0:  # gets closest station
            selected_rows.append(row)
            selected_xy.append(cand_xy)
        else:
            angles = [
                compute_angle(cand_xy - center_xy, s - center_xy) for s in selected_xy
            ]
            min_angle = min(angles)
            if min_angle > np.radians(45) or len(selected_xy) < k // 2:
                selected_rows.append(row)
                selected_xy.append(cand_xy)
        if len(selected_rows) == k:
            break

    # Order by East -> West -> North/South
    # Convert selected_rows to DataFrame
    result_df = pd.DataFrame(selected_rows).reset_index(drop=True)
    result_df = getEastWestNorthSouthOrder(result_df)

    return result_df, center_latlon


def find_nearest_station_given_long_lat(df, Longitude, Latitude):
    # Find the station with the smallest Euclidean distance to the given coordinates
    df["distance"] = df.apply(
        lambda row: euclidean_distance(
            Latitude, Longitude, row["Latitude"], row["Longitude"]
        ),
        axis=1,
    )
    nearest_station = df.sort_values(
        by="distance", key=lambda x: x.apply(lambda y: y[0])
    ).head(1)
    return nearest_station[
        ["station", "Latitude", "Longitude", "distance", "StartTime", "EndTime"]
    ]

    # To get station name print(str(wanted_station.station.iloc[0]))


def find_nearest_station_given_normed_long_lat(df, Longitude_Normed, Latitude_Normed):
    # Find the station with the smallest Euclidean distance to the given coordinates
    df["distance"] = df.apply(
        lambda row: euclidean_distance(
            Longitude_Normed, Latitude_Normed, row["norm_lat"], row["norm_long"]
        ),
        axis=1,
    )
    nearest_station = df.sort_values(
        by="distance", key=lambda x: x.apply(lambda y: y[0])
    ).head(1)
    return nearest_station[
        ["station", "norm_lat", "norm_long", "distance", "StartTime", "EndTime"]
    ]

    # To get station name print(str(wanted_station.station.iloc[0]))


def modify_nearest_stations(nearest_stations):
    modified_nearest_stations = []
    for station in nearest_stations["station"]:
        modified_station = station.replace(" ", "-")
        modified_nearest_stations.append(modified_station)
    return modified_nearest_stations


def find_stations_csv(stations, csvs):
    station_order = []
    nearest_stations_csvs = []
    for station in stations:
        for csv in csvs:
            if station.lower().replace("-", "_") in csv.lower().replace("-", "_"):
                nearest_stations_csvs.append(csv)
                print(f"Station {station} found in {csv}")
                station_order.append(station)
                break
    return nearest_stations_csvs, station_order


def adjustWindDirection(wind_direction_degress, theta_relative):
    # Adjusts auxillary stations wind direction to be relative to the wanted stations position
    # Adjust the wind direction by subtracting the relative angle
    adjusted_wind_direction = wind_direction_degress - theta_relative
    # Normalize to 0-360 degrees
    adjusted_wind_direction = (adjusted_wind_direction + 360) % 360
    return adjusted_wind_direction


def cyclicalEncoding(data, cycleLength):
    newDatasin = np.sin(
        2 * np.pi * (data - 1) / cycleLength
    )  # Sine encoding for hours (adjust for 0-23)
    newDatacos = np.cos(
        2 * np.pi * (data - 1) / cycleLength
    )  # Cosine encoding for hours (adjust for 0-23)
    return newDatasin, newDatacos


def process_data(
    df,
    usecols,
    min_start_year,
    max_end_year,
    i,
    wantedStationCSV=False,
    RelativeAnglesDegrees=[(0, 0)],
):
    for col in usecols:
        match col:
            case x if x == COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"]:
                # Convert columns to numeric, coercing errors to NaN
                df[col] = pd.to_numeric(df[col], errors="coerce").replace(99, np.nan)
                df = df[df[col].between(min_start_year, max_end_year)]
                df = df.drop(columns=[col], axis=1)

            case x if x == COLUMN_NAMES["OPAQUE_SKY_COVER"]:
                df[col] = pd.to_numeric(df[col], errors="coerce").replace(99, np.nan)

            case x if x in [COLUMN_NAMES["MONTH"], COLUMN_NAMES["HOUR"]]:
                df = df.drop(columns=[col], axis=1)

            case x if x in [
                COLUMN_NAMES["HOUR_SIN"],
                COLUMN_NAMES["HOUR_COS"],
                COLUMN_NAMES["MONTH_SIN"],
                COLUMN_NAMES["MONTH_COS"],
            ]:
                if not wantedStationCSV:
                    df = df.drop(columns=[col], axis=1)
                else:
                    df[col] = pd.to_numeric(df[col], errors="coerce").replace(
                        99, np.nan
                    )

            case x if x in [
                COLUMN_NAMES["WIND_DIRECTION_SIN"],
                COLUMN_NAMES["WIND_DIRECTION_COS"],
            ]:
                if wantedStationCSV:
                    df[col] = pd.to_numeric(df[col], errors="coerce").replace(
                        99, np.nan
                    )

            case x if x == COLUMN_NAMES["WIND_DIRECTION"]:
                if not wantedStationCSV:
                    df[col] = pd.to_numeric(df[col], errors="coerce").replace(
                        99, np.nan
                    )
                    df[col] = df[col].apply(
                        lambda x: adjustWindDirection(x, RelativeAnglesDegrees[i])
                    )
                    radians = np.deg2rad(df[col])
                    df[COLUMN_NAMES["WIND_DIRECTION_SIN"]] = np.sin(radians)
                    df[COLUMN_NAMES["WIND_DIRECTION_COS"]] = np.cos(radians)
                df = df.drop(columns=[col], axis=1)

            case x if x == COLUMN_NAMES["WIND_SPEED"]:
                df[col] = pd.to_numeric(df[col], errors="coerce").replace(99, np.nan)

            case x if x in [
                COLUMN_NAMES["GHI"],
                COLUMN_NAMES["DNI"],
                COLUMN_NAMES["DHI"],
            ]:
                df[col] = pd.to_numeric(df[col], errors="coerce").replace(9999, np.nan)

            case _:
                df[col] = df.drop(columns=[col], axis=1)
    return df


def get_nearest_stations_data(
    nearest_stations_csvs,
    min_start_year,
    max_end_year,
    wantedStationCSV=False,
    RelativeAnglesDegrees=[(0, 0)],
    usecols=[],
    dtype={},
):
    dfs = []
    i = 0
    for f in nearest_stations_csvs:
        # Read the CSV file with specified columns and data types
        df = pd.read_csv(
            f,
            delimiter=",",
            index_col=False,
            usecols=usecols,
            dtype=dtype,
            on_bad_lines="skip",
        )

        df = process_data(
            df,
            usecols,
            min_start_year,
            max_end_year,
            i,
            wantedStationCSV=wantedStationCSV,
            RelativeAnglesDegrees=RelativeAnglesDegrees,
        )

        # print(df.dtypes)
        df.fillna(0, inplace=True)
        dfs.append(df)
        i += 1
        # print(df.columns)
    return dfs


def chunk(df, interval=25):
    results = []
    for i in range(0, len(df) - interval, 1):
        chunk = df[i : i + interval]
        results.append(chunk)
    results = np.array(results)
    return results


def get_chunked_tensors(nearest_stations, dfs, interval):
    chunked_tensors = []
    rows = 0
    station_order = []
    for index, station in nearest_stations.iterrows():
        distanceVector = station.iloc[3][1]
        chunked_df = chunk(dfs[rows], interval=interval)
        rows += 1
        # print("chunked df shape:", chunked_df.shape)
        chunkedTensor = torch.tensor(chunked_df).to(torch.float32)
        chunked_tensors.append(chunkedTensor)
        station_order.append(station)
    return chunked_tensors, station_order


def getMaxStartMinEndYearComplete(max_start_year, min_end_year):
    maxStartYear, minEndYear = str(max_start_year), str(min_end_year)
    # print(max_start_year, min_end_year)
    maxStartYear = maxStartYear + "010100"
    minEndYear = minEndYear + "123124"
    maxStartYear = int(maxStartYear)
    minEndYear = int(minEndYear)
    return maxStartYear, minEndYear


# def angle_from_wanted_station(row, wanted_station_long, wanted_station_lat):
#   print(row["station"])
#   lat, lon = row['Latitude'], row['Longitude']
#   delta_lon = lon - wanted_station_long
#   delta_lat = lat - wanted_station_lat
#   angle = np.arctan2(delta_lat, delta_lon)  # Angle in radians
#   angle_degrees = np.degrees(angle)
#   if angle_degrees < 0:
#     angle_degrees += 360  # Normalize to [0, 360]
#   return angle_degrees

# def sortAuxillaryStations(nearest_stations_df, wanted_station_lat, wanted_station_long):
#   """
#   From the auxillary stations found, sort them to have the first one be the eastern most station, the second be the western most station the third be the northern most station and the fourth be the southern most station
#   """

#   nearest_stations_df['angle'] = nearest_stations_df.apply(angle_from_wanted_station, axis=1, wanted_station_long=wanted_station_long, wanted_station_lat=wanted_station_lat)
#   sorted_stations = nearest_stations_df.sort_values(by='angle').reset_index(drop=True)
#   print("Sorted Stations by angle (0 degrees is east, 90 degrees is north, 180 degrees is west, 270 degrees is south):")
#   print(sorted_stations.head())
#   return sorted_stations


def normalize_column(column: str, wanted_df: pd.DataFrame, aux_dfs: list[pd.DataFrame]):
    # Calculate mean and std across all dataframes. Can use mean of means as they all have same length
    wanted_mean = wanted_df[column].mean()
    wanted_std = wanted_df[column].std()

    aux_means = [df[column].mean() for df in aux_dfs]
    aux_stds = [df[column].std() for df in aux_dfs]

    mean = (wanted_mean + sum(aux_means)) / (1 + len(aux_means))
    std = (wanted_std + sum(aux_stds)) / (1 + len(aux_stds))

    # Standardized Normalization
    # Normalizing wanted station
    wanted_df[column] = (wanted_df[column] - mean) / std
    # Normalizing auxillary stations
    for df in aux_dfs:
        df[column] = (df[column] - mean) / std

    return wanted_df, aux_dfs, mean, std


columns_to_normalize = [
    COLUMN_NAMES["GHI"],
    COLUMN_NAMES["DNI"],
    COLUMN_NAMES["DHI"],
    COLUMN_NAMES["WIND_SPEED"],
]


def normalize_dataframes(wanted_df: pd.DataFrame, aux_dfs: list[pd.DataFrame]):
    meanGHI = 0
    stdGHI = 0
    for column in columns_to_normalize:
        wanted_df, aux_dfs, mean, std = normalize_column(column, wanted_df, aux_dfs)
        print(f"Normalized {column} with mean: {mean}, std: {std}")
        if column == COLUMN_NAMES["GHI"]:
            meanGHI = mean
            stdGHI = std

    return [wanted_df], aux_dfs, meanGHI, stdGHI


def get_relative_angle_degrees(nearest_stations):
    RelativeAnglesDegrees = []
    for distanceVector in nearest_stations["distance"].values:
        Relative_Angle_Radians = np.arctan2(
            distanceVector[1][1], distanceVector[1][0]
        )  # Relative Distance Vector y and Distance Vector x
        Relative_Angle_Degrees = np.degrees(Relative_Angle_Radians)
        if Relative_Angle_Degrees < 0:
            Relative_Angle_Degrees += 360
        RelativeAnglesDegrees.append(Relative_Angle_Degrees)
    return RelativeAnglesDegrees


def getAllKAuxillaryStationsReadyByWantedStationName(
    stationsName_lat_long_datadf,
    nearest_stations,
    k,
    min_start_year,
    max_end_year,
    RelativeAnglesDegrees,
    csv_files,
    usecols=[],
    dtype={},
):
    # doing all thats needed to get Auxillary stations data ready and in chunked tensors and in order chunked
    modified_nearest_stations = modify_nearest_stations(nearest_stations)
    nearest_stations_csvs, station_order = find_stations_csv(
        modified_nearest_stations, csv_files
    )
    nearest_stations_data_dfs = get_nearest_stations_data(
        nearest_stations_csvs=nearest_stations_csvs,
        min_start_year=min_start_year,
        max_end_year=max_end_year,
        wantedStationCSV=False,
        RelativeAnglesDegrees=RelativeAnglesDegrees,
        usecols=usecols,
        dtype=dtype,
    )
    for i in range(k):
        nearest_stations_data_dfs[i]["distanceX"] = [
            nearest_stations["distance"].values[i][1][0]
        ] * len(nearest_stations_data_dfs[i])
        nearest_stations_data_dfs[i]["distanceY"] = [
            nearest_stations["distance"].values[i][1][1]
        ] * len(nearest_stations_data_dfs[i])
    return nearest_stations_data_dfs, nearest_stations_data_dfs


def getAllReadyForStationByLatAndLongAndK(
    stationsName_lat_long_datadf, lat, long, k, csv_files, usecols=[], dtype={}
):
    try:
        # doing everything needed to get wanted stations data ready and in chunked tensors as well as get wanted station and its name
        wanted_station = find_nearest_station_given_long_lat(
            stationsName_lat_long_datadf, lat, long
        )
        max_start_year = wanted_station["StartTime"].values[0]
        min_end_year = wanted_station["EndTime"].values[0]

        wanted_station.at[wanted_station.index[0], "distance"] = (0.0, (0.0, 0.0))
        wanted_station_modified = modify_nearest_stations(
            {"station": wanted_station["station"]}
        )
        wanted_station_csv, _ = find_stations_csv(wanted_station_modified, csv_files)
        nearest_stations, target_point = spatially_diverse_knn(
            stationsName_lat_long_datadf, wanted_station["station"].values[0], k
        )

        max_start_year, min_end_year = (
            max(max_start_year, *nearest_stations["StartTime"].values),
            min(min_end_year, *nearest_stations["EndTime"].values),
        )
        max_start_year, min_end_year = getMaxStartMinEndYearComplete(
            max_start_year, min_end_year
        )
        print(max_start_year, min_end_year)

        # Get Relative Angles in Degrees for Wind Direction Adjustment
        RelativeAnglesDegrees = get_relative_angle_degrees(nearest_stations)

        # Get Auxillary Stations DataFrames
        nearest_stations_data_dfs, aux_stations_dfs = (
            getAllKAuxillaryStationsReadyByWantedStationName(
                stationsName_lat_long_datadf=stationsName_lat_long_datadf,
                nearest_stations=nearest_stations,
                k=k,
                min_start_year=max_start_year,
                max_end_year=min_end_year,
                RelativeAnglesDegrees=RelativeAnglesDegrees,
                csv_files=csv_files,
                usecols=usecols,
                dtype=dtype,
            )
        )

        # Get Wanted Station DataFrame
        wanted_station_data_dfs = get_nearest_stations_data(
            wanted_station_csv,
            max_start_year,
            min_end_year,
            wantedStationCSV=True,
            usecols=usecols,
            dtype=dtype,
        )

        # Normalize all stations data.
        wanted_station_data_dfs, aux_stations_dfs, meanGHI, stdGHI = (
            normalize_dataframes(wanted_station_data_dfs[0], aux_stations_dfs)
        )

        # Get Auxillary Stations Chunked Tensors
        aux_chunked_tensors, aux_chunked_station_order = get_chunked_tensors(
            nearest_stations, nearest_stations_data_dfs, 25
        )

        # Get Wanted Station Chunked Tensors
        wanted_chunked_tensors, _ = get_chunked_tensors(
            wanted_station, wanted_station_data_dfs, 25
        )

        return (
            wanted_chunked_tensors,
            wanted_station["station"].values[0],
            aux_chunked_tensors,
            aux_chunked_station_order,
            meanGHI,
            stdGHI,
            aux_stations_dfs,
            wanted_station_data_dfs,
        )

    except Exception as e:
        print("Error in getAllReadyForStationByLatAndLongAndK: ", e)
        return None, None, None, None, None, None, None, None


def getAllReadyForStationByLatAndLongAndKSplitTestAndTrain(
    stationsName_lat_long_datadf, lat, long, k, csv_files, usecols=[], dtype={}
):
    # doing everything needed to get wanted stations data ready and in chunked tensors as well as get wanted station and its name
    wanted_station = find_nearest_station_given_long_lat(
        stationsName_lat_long_datadf, lat, long
    )
    max_start_year = wanted_station["StartTime"].values[0]
    min_end_year = wanted_station["EndTime"].values[0]

    wanted_station.at[wanted_station.index[0], "distance"] = (0.0, (0.0, 0.0))
    wanted_station_modified = modify_nearest_stations(
        {"station": wanted_station["station"]}
    )
    wanted_station_csv, _ = find_stations_csv(wanted_station_modified, csv_files)

    nearest_stations, target_point = spatially_diverse_knn(
        stationsName_lat_long_datadf, wanted_station["station"].values[0], k
    )
    # plot_stations_matplotlib(target_point, nearest_stations, stationsName_lat_long_datadf)
    max_start_year, min_end_year = (
        max(max_start_year, *nearest_stations["StartTime"].values),
        min(min_end_year, *nearest_stations["EndTime"].values),
    )  # Get earliest year that all start at (max_start_year) and latest year all go until (min_end_year)
    train_max_start_year, train_min_end_year = getMaxStartMinEndYearComplete(
        max_start_year, min_end_year - 2
    )
    test_max_start_year, test_min_end_year = getMaxStartMinEndYearComplete(
        min_end_year - 1, min_end_year
    )

    print(max_start_year, min_end_year)
    print(train_max_start_year, train_min_end_year)
    print(test_max_start_year, test_min_end_year)

    RelativeAnglesDegrees = []
    for distanceVector in nearest_stations["distance"].values:
        Relative_Angle_Radians = np.arctan2(
            distanceVector[1][1], distanceVector[1][0]
        )  # Relative Distance Vector y and Distance Vector x
        Relative_Angle_Degrees = np.degrees(Relative_Angle_Radians)
        if Relative_Angle_Degrees < 0:
            Relative_Angle_Degrees += 360
        RelativeAnglesDegrees.append(Relative_Angle_Degrees)

    trainSet_aux_chunked_tensors, trainSet_aux_chunked_station_order = (
        getAllKAuxillaryStationsReadyByWantedStationName(
            stationsName_lat_long_datadf,
            nearest_stations,
            k,
            train_max_start_year,
            train_min_end_year,
            RelativeAnglesDegrees,
            csv_files,
            usecols=usecols,
            dtype=dtype,
        )
    )
    testSet_aux_chunked_tensors, testSet_aux_chunked_station_order = (
        getAllKAuxillaryStationsReadyByWantedStationName(
            stationsName_lat_long_datadf,
            nearest_stations,
            k,
            test_max_start_year,
            test_min_end_year,
            RelativeAnglesDegrees,
            csv_files,
            usecols=usecols,
            dtype=dtype,
        )
    )

    trainSet_wanted_station_data_dfs = get_nearest_stations_data(
        wanted_station_csv,
        train_max_start_year,
        train_min_end_year,
        wantedStationCSV=True,
        usecols=usecols,
        dtype=dtype,
    )
    testSet_wanted_station_data_dfs = get_nearest_stations_data(
        wanted_station_csv,
        test_max_start_year,
        test_min_end_year,
        wantedStationCSV=True,
        usecols=usecols,
        dtype=dtype,
    )
    # print(wanted_station_data_dfs[0].head())

    trainSet_wanted_chunked_tensors, _ = get_chunked_tensors(
        wanted_station, trainSet_wanted_station_data_dfs, 25
    )
    testSet_wanted_chunked_tensors, _ = get_chunked_tensors(
        wanted_station, testSet_wanted_station_data_dfs, 25
    )

    return (
        trainSet_wanted_chunked_tensors,
        testSet_wanted_chunked_tensors,
        wanted_station["station"].values[0],
        trainSet_aux_chunked_tensors,
        testSet_aux_chunked_tensors,
        trainSet_aux_chunked_station_order,
        testSet_aux_chunked_station_order,
    )


num_aux_stations = 4

## model related

In [ ]:
def test_model(model, test_loader, criterion, device):
    # if(combined_test_chunked_data_tensor.shape):
    #   dataset = GHIDataset(combined_test_chunked_data_tensor, device=device)
    #   train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    # dataset = GHIDataset(combined_chunked_data_tensor, device=device)
    # train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    model.eval()
    outputlist = []
    targetData = []
    colourList = []
    for inputs, targets in test_loader:
        output = model(inputs)
        colourList.append(
            inputs[:, -1].detach().cpu().numpy()
        )  # Saving hour before predicted hour
        targetData.append(targets.detach().cpu().numpy())
        outputlist.append(output.detach().cpu().numpy())
    colourList = np.concatenate(
        colourList[:-1], axis=0
    )  # Last batch size is too small so getting rid of it with the -1
    outputlist = np.concatenate(outputlist[:-1], axis=0)
    targetData = np.concatenate(targetData[:-1], axis=0)
    # del combined_chunked_data_tensor #Saving RAM space
    return outputlist, targetData, colourList

# Model Declaration

In [ ]:
class GHIDataset(Dataset):
    def __init__(self, chunked_data, device="cuda"):
        super(GHIDataset, self).__init__()
        self.chunked_data = chunked_data.to(
            device
        )  # chunked data needs to be passed in as a tensor
        self.device = device

    def __len__(self):
        return len(self.chunked_data)

    def __getitem__(self, i):
        element = self.chunked_data[
            i
        ]  # need to break down to zeroth element of list and first element of that is GHI of that which is the target
        # inputs = []
        # for el in element:
        #   inputs.append(el[0:-1])
        # print(element.shape)
        return element[0:-1], element[
            -1, 0
        ]  # ensure first station in dataset is the one we are wanting to predict


class Main_LSTM(nn.Module):
    def __init__(self, dropout=0.0, num_aux_stations=3):
        """
        input_dim: Number of features per time step (here, 5)
        hidden_dim: Number of hidden units in the LSTM
        num_layers: Number of stacked LSTM layers
        dropout: Dropout probability (applied between LSTM layers if num_layers > 1)
        """
        super(Main_LSTM, self).__init__()
        num_input = 10 + num_aux_stations * 8
        self.Stationlstm = nn.LSTM(num_input, 128, 3, batch_first=True, dropout=dropout)
        self.Auxlstm = nn.LSTM(
            8, 8, 2, batch_first=True, dropout=dropout
        )  # may need to change 16 back to 8
        # A fully-connected layer to map the LSTM output to a single GHI prediction
        self.station_fc = nn.Linear(128, 1)
        self.num_aux_stations = num_aux_stations

    def forward(self, x):
        # x shape: (batch_size, sequence_length, input_dim)
        # LSTM returns output for every time step, and the final hidden and cell states.

        # torch.Size([32, 24, 64])
        main_station_data = x[:, :, :10]  # First 10 features are for main station
        aux_station_data = x[:, :, 10:]  # Remaining features are for auxiliary stations
        aux_lstm_outs = []
        # other idea: range start at 10 go to 34 add 8 each time as 3 auxillaries
        for i in range(
            0, self.num_aux_stations * 8, 8
        ):  # range start at 0 go to 8*num_aux_stations add 8 each time as num_aux_stations auxillary stations
            lstm_out, _ = self.Auxlstm(aux_station_data[:, :, i : i + 8])
            aux_lstm_outs.append(lstm_out)
        concat = torch.cat(([main_station_data] + aux_lstm_outs), 2)
        # print(concat.shape)
        lstm_out, (h_n, c_n) = self.Stationlstm(concat)
        # Use the output from the last time step as a summary of the sequence
        # lstm_out[:, -1, :] has shape (batch_size, hidden_dim)
        # print("Main: ", lstm_out.shape)
        final_feature = lstm_out[:, -1, :]  # Not sure if this WORKSSSS (Right size?)
        # torch.Size([32, 64])

        # Pass through the fully-connected layer to produce a single output
        ghi_pred = self.station_fc(final_feature)  # shape: (batch_size, 1)
        return ghi_pred

In [ ]:
device = "cpu"  # torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
mainModel = Main_LSTM(num_aux_stations=num_aux_stations).to(device)
mainModel = torch.compile(mainModel)
# mainModel = Main_LSTM()
criterion = nn.MSELoss()  # Mean Squared Error is common for regression tasks
optimizer_main = optim.Adam(mainModel.parameters(), lr=0.001)

Using device: cpu


# Testing

In [26]:
non_cloud_path = "Datasets/CWEEDS_2020_BC_cleaned_non_cloud"
cloud_path = "Datasets/CWEEDS_2020_BC_cleaned_cloud"
csv_files = glob.glob(os.path.join(non_cloud_path, "*.csv"))
file_path = "Datasets/stationsName_lat_long_data.csv"
stationsName_lat_long_datadf = pd.read_csv(
    file_path, delimiter=",", on_bad_lines="skip"
)
num_aux_stations = 4
(
    wanted_chunked_tensors,
    wanted_station_name,
    aux_chunked_tensors,
    aux_chunked_station_order,
    meanGHI1,
    stdGHI1,
    aux_stations_dfs,
    wanted_station_data_dfs,
) = getAllReadyForStationByLatAndLongAndK(
    stationsName_lat_long_datadf.copy(),
    lat=-122.5,
    long=53,
    k=num_aux_stations,
    csv_files=csv_files,
    usecols=USECOLS_NON_CLOUD,
    dtype=DTYPE_NON_CLOUD,
)
print("combining")
combined_train_chunked_data_tensor = torch.cat(
    wanted_chunked_tensors + aux_chunked_tensors, dim=2
)

Station QUESNEL found in Datasets/CWEEDS_2020_BC_cleaned_non_cloud\QUESNEL_no_sky_cover.csv
1998010100 2017123124
Station BLUE-RIVER-CS found in Datasets/CWEEDS_2020_BC_cleaned_non_cloud\BLUE_RIVER_CS_no_sky_cover.csv
Station PUNTZI-MOUNTAIN-(AUT) found in Datasets/CWEEDS_2020_BC_cleaned_non_cloud\PUNTZI_MOUNTAIN_(AUT)_no_sky_cover.csv
Station PRINCE-GEORGE found in Datasets/CWEEDS_2020_BC_cleaned_non_cloud\PRINCE_GEORGE_no_sky_cover.csv
Station WILLIAMS-LAKE-A found in Datasets/CWEEDS_2020_BC_cleaned_non_cloud\WILLIAMS_LAKE_A_no_sky_cover.csv
got auxillary stations data dfs
<class 'list'> <class 'pandas.core.frame.DataFrame'>
Normalized GHI_kJ/m2 with mean: 478.9616278804472, std: 753.7647783590521
Normalized DNI_kJ/m2 with mean: 452.87493725758605, std: 847.7786302752787
Normalized DHI_kJ/m2 with mean: 249.06477983116588, std: 356.09061815781496
Normalized wind_speed with mean: 20.444437599817476, std: 20.070777452990047
Normalized all dataframes
<class 'list'> <class 'pandas.core.fr

In [27]:
combined_train_chunked_data_tensor.shape

torch.Size([175295, 25, 42])

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt


def analyze_auxiliary_stations(wanted_df, aux_list, lags=24, alpha=0.01, plot=True):
    """
    Compare multiple auxiliary stations' wind features against wanted station GHI.

    Parameters
    ----------
    wanted_df : pd.DataFrame
        Must contain 'timestamp' and 'GHI_target'.
    aux_list : list of pd.DataFrame
        Each dataframe must contain 'timestamp', 'wind_speed', 'wind_direction'.
    lags : int
        Number of lag hours to test (default 24).
    alpha : float
        Regularization strength for Lasso when testing multi-lags.
    plot : bool
        If True, plots R² vs lag for each station.

    Returns
    -------
    results : dict
        Keys = station index, values = dict with:
            - 'best_single_lag'
            - 'lag_ranking' (DataFrame of lag & R²)
            - 'best_multi_lags' (list from Lasso)
    """

    results = {}
    print(wanted_df.head())
    for idx, aux_df in enumerate(aux_list):
        print(aux_df.head())
        # --- 1. Merge wanted + auxiliary station on timestamp ---
        df = pd.merge(
            wanted_df[[COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"], COLUMN_NAMES["GHI"]]],
            aux_df[
                [
                    COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"],
                    COLUMN_NAMES["WIND_SPEED"],
                    COLUMN_NAMES["WIND_DIRECTION"],
                ]
            ],
            on=COLUMN_NAMES["YEAR_MONTH_DAY_HOUR"],
            how="inner",
        )

        # --- 2. Build lagged features ---
        X = pd.DataFrame()
        for lag in range(1, lags + 1):
            X[f"wind_speed_lag_{lag}"] = df[COLUMN_NAMES["WIND_SPEED"]].shift(lag)
            X[f"wind_dir_lag_{lag}"] = df[COLUMN_NAMES["WIND_DIRECTION"]].shift(lag)

        y = df["GHI_target"]

        # Drop missing
        data = pd.concat([X, y], axis=1).dropna()
        X = data.drop(columns=COLUMN_NAMES["GHI"], axis=1)
        y = data[COLUMN_NAMES["GHI"]]

        # --- 3. Single-lag regressions ---
        r2_scores = []
        for lag in range(1, lags + 1):
            X_lag = data[[f"wind_speed_lag_{lag}", f"wind_dir_lag_{lag}"]]
            model = LinearRegression()
            model.fit(X_lag, y)
            y_pred = model.predict(X_lag)
            r2_scores.append(r2_score(y, y_pred))

        best_single_lag = int(np.argmax(r2_scores) + 1)

        lag_ranking = (
            pd.DataFrame({"lag_hour": list(range(1, lags + 1)), "r2_score": r2_scores})
            .sort_values(by="r2_score", ascending=False)
            .reset_index(drop=True)
        )

        # --- 4. Multi-lag via Lasso ---
        lasso = LassoCV(alphas=[alpha], cv=5).fit(X, y)
        coef_mask = np.abs(lasso.coef_) > 1e-6
        best_multi_lags = list(X.columns[coef_mask])

        # --- 5. Optional plot ---
        if plot:
            plt.figure(figsize=(10, 4))
            plt.plot(range(1, lags + 1), r2_scores, marker="o")
            plt.xlabel("Lag hour")
            plt.ylabel("R² score")
            plt.title(f"Aux Station {idx + 1}: Predictive power of wind (speed+dir)")
            plt.grid(True)
            plt.show()

        # --- 6. Save results ---
        results[idx] = {
            "best_single_lag": best_single_lag,
            "lag_ranking": lag_ranking,
            "best_multi_lags": best_multi_lags,
        }

    return results


In [ ]:
# wanted_df has: timestamp, GHI_target
# aux_list = [aux1_df, aux2_df, aux3_df, aux4_df],
results = analyze_auxiliary_stations(wanted_station_data_dfs[0], aux_stations_dfs)

# Show best lags
for i, res in results.items():
    print(f"\nAux Station {i + 1}")
    print("  Best single lag:", res["best_single_lag"])
    print("  Top 3 lag hours:\n", res["lag_ranking"].head(3))
    print("  Best multi-hour combo:", res["best_multi_lags"])


   GHI_kJ/m2  DNI_kJ/m2  DHI_kJ/m2  wind_speed  Hour_sin  Hour_cos  Month_sin  \
0          0          0          0    1.919726  0.000000  1.000000        0.0   
1          0          0          0    1.204601  0.258819  0.965926        0.0   
2          0          0          0    1.460003  0.500000  0.866025        0.0   
3          0          0          0    1.204601  0.707107  0.707107        0.0   
4          0          0          0    2.737011  0.866025  0.500000        0.0   

   Month_cos  wind_dir_sin  wind_dir_cos  
0        1.0     -0.173648      0.984808  
1        1.0     -0.500000      0.866025  
2        1.0      0.173648      0.984808  
3        1.0      0.342020      0.939693  
4        1.0     -0.173648      0.984808  
   GHI_kJ/m2  DNI_kJ/m2  DHI_kJ/m2  wind_speed  wind_dir_sin  wind_dir_cos  \
0          0          0          0   -0.272811      0.829075      0.559137   
1          0          0          0   -0.619139     -0.587840     -0.808977   
2          0         

KeyError: "['Year Month Day Hour (YYYYMMDDHH)'] not in index"

In [ ]:
for df in aux_stations_dfs:
    df["GHI_kJ/m2"] = wanted_station_data_dfs[0]["GHI_kJ/m2"].values

aux_stations_df = pd.concat(aux_stations_dfs, axis=0).reset_index(drop=True)

In [ ]:
data = engineer_features(aux_stations_df.copy())
X, y = prepare_features(data)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = train_distance_model(X_train, y_train)
evaluate_distance_model(model, X_test, y_test)

RMSE on test set: 694.38 W/m^2
Feature importances:
Wind speed / 0.1 m/s    0.458712
alignment               0.189144
wind_u                  0.148203
distance                0.116659
wind_v                  0.087282
dtype: float64
